In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig, HttpOptions
from google.genai import types
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from __future__ import annotations

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

In [ ]:
# CLient setup
# This work rely on the google API to use Gemini 2.5 and gemini-embedding-001
# Add your API key here to call the models
API_KEY = 

NB_LAYER = 5
client = genai.Client(api_key=API_KEY)

# Text generation function

In [ ]:
#################################################################################################################################################################################################################################
# Class and prompts definitions
#################################################################################################################################################################################################################################

from enum import Enum
from pydantic import BaseModel

# Config file
###############################################################################################

# The Config class define the content of the config file that must be saved alongside any experiment results
# It contains a description of the experimental parameters/setup used in the experiment

class Config:
    model : str
    output_per_prompt : int
    layer_iteration  : int
    final_failure_count : int
    total_failure_count : int
    model_heat : int
    sys_prompt :str
    # prompt : str

    def __init__(self, sys_prompt, layer_iteration, output_per_prompt, final_failure_count, total_failure_count, model_heat = 1, model = "gemini-2.5-flash", ):
        self.model = model
        self.output_per_prompt = output_per_prompt
        self.layer_iteration = layer_iteration
        self.final_failure_count = final_failure_count
        self.total_failure_count = total_failure_count
        self.model_heat = model_heat
        self.sys_prompt = sys_prompt
        # self.prompt = prompt

    def __str__(self):
        return  f"model: {self.model}\noutput_per_prompt: {self.output_per_prompt}\nlayer_iteration: {self.layer_iteration}\nfinal_failure_count: {self.final_failure_count}\ntotal_failure_count: {self.total_failure_count}\nmodel_heat: {self.model_heat}\nsys_prompt: {self.sys_prompt}\n"


# Structured scenario 
###############################################################################################


# Defintion of layer 1
class Guidance_type(str, Enum):
    marking = "marking"
    sign = "sign"
    light = "traffic light"
    other = "other"

class Road_type(str, Enum):
    highway = "highway"
    road = "road"
    bike_lane = "bike_lane"
    bus_lane = "bus_lane"
    train_tracks = "train_tracks"
    tram_lane = "tram_lane"
    parking = "parking"


class Road_geometry(str, Enum):
    straight = "straight"
    turning_left = "turning left"
    turning_right = 'turning right'
    round_about = "round about"
    hairpin_turn = "hairpin turn"
    other = "other"


class Road(BaseModel):
    type : Road_type
    geometry : Road_geometry
    main_road : bool
    total_number_of_lanes_in_both_direction : int
    lane_size : float
    surface_material : str
    eventual_damages : str

class Guidance_object(BaseModel):
    type: Guidance_type
    characteristics: str
    position: str


class Road_layer(BaseModel):
    roads : list[Road]
    guidance_objects : list[Guidance_object]

# Defintion of layer 2


# class Environment(Enum):

class Structure_type(str, Enum):
    skyscraper = "skyscraper"
    multy_story_building = "multy-story building"
    small_building = "small building"
    wall_or_barrier = "wall or barrier"
    vegetation = "vegetation"
    water = "surface of water"
    bridge = "bridge"
    tunnel = "tunnel"
    cliff = "cliff"
    miscellaneous = "miscellaneous man-made structure"
    other = "other"

class Environment_type(str, Enum):
    urban = "urban"
    rural = "rural"
    forest = "forest"
    desertic = "desertic"
    mountainous = "mountainous"
    waterside = "seaside or lakeside"
    other = "other"

class Structure(BaseModel):
    type: Structure_type
    characteristics: str
    position: str

class Surrounding_structure_layer(BaseModel):
    environment : Environment_type
    structures : list[Structure]


# Defintion of layer 3

class Temporary_change(BaseModel):
    type : str
    characteristic : str
    position : str

class Temporary_changes_layer(BaseModel):
    change_list : list[Temporary_change]

# Definition of layer 4

class Dynamic_object_type(str, Enum):
    vehicle = "Vehicle"
    cyclist = "cyclist"
    pedestrian = "pedestrian"
    animal = "animal"
    robot = "robot"
    inanimate_object = "inanimate_object"
    liquid = "liquid"
    smoke = "smoke"
    other = "other"

class Motion_type(str, Enum):
    not_moving = "not moving"
    moving_toward_ego = "moving toward ego"
    moving_in_same_direction_as_ego = "moving in same direction as ego"
    moving_in_opposite_direction_as_ego = "moving in same direction as ego"
    moving_erratically = "moving erratically"
    falling = "falling"
    other = "other"

class Dynamic_object(BaseModel):
    type : Dynamic_object_type
    characteristic : str
    position : str
    motion_type: Motion_type
    motion_characteristic: str

class Dynamic_object_layer(BaseModel):
    object_list : list[Dynamic_object]

# Definition of layer 5

class Weather_type(str, Enum):
    clear = "clear"
    cloudy = "cloudy"
    raining = "raining"
    storm = "storm"
    snowing = "snowing"
    foggy = "foggy"
    other = "other"

class Weather(BaseModel):
    type: Weather_type
    characteristics : str

class Light_source(str, Enum):
    sunlight = "sunlight"
    lamps = "lamps"
    neon = "neon"
    screen = "screen"
    vehicle_light = "vehicle_light"
    reflection = "reflection"
    other = "other"

class Illumination(BaseModel):
    light_source : Light_source
    characteristic : str

class Environment_layer(BaseModel):
    weather : Weather
    illumination : list[Illumination]

# Final class

class Scenario(BaseModel):
    L1 : Road_layer
    L2 : Surrounding_structure_layer
    L3 : Temporary_changes_layer
    L4 : Dynamic_object_layer
    L5 : Environment_layer

# For output_per_prompt >1 :

class Scenario_list(BaseModel):
    output : list[Scenario]



# Prompt definition
###############################################################################################

# System prompts #######################


sys_prompt_segment__Role = "# Role: You are a driving scenario generator. Your purpose is to generate new confusing and challenging Edge Case scenarios from the input scenario.\n"

sys_prompt_segment__Layer_model = f"# Format: Your output must follow the {NB_LAYER} layer model of the input scenario description where:\n" \
"1. The first layer contains the road network and traffic guidance objects. It specifies the number of lanes as well as the presence of road marking, signs, parking space...\n" \
"2. The second layer contains roadside structures, which include any static objects outside of the road, such as building, vegetation or guardrails.\n" \
"3. The third layer contains any temporary modification to the layers 1 and 2. It includes roadworks, covered marking, static obstacles on the road or holes.\n" \
"4. The fourth layer contains every dynamic and moveable objects, like vehicles, pedestrians, animals, miscellaneous objects or moving structures.\n" \
"5. The fifth layer contains the environmental conditions of the scene, including road weather (dry, wet, icy...), illumination and eventual precipitation.\n" \
"For each layer, your textual description must be concise, but as exhaustive as possible.\n" \
"For the fourth layer in particular, define each component in relation to the ego vehicule.\n" \
"- Please respect the following guidelines when generating a new scenario (IMPORTANT):\n"\
"1) Layers 1, 2 and 3 conduct a spatial-based description. They do not contain any time-variable aspects. Time-based descriptions are introduced from Layer 4 upwards.\n"\
"2) Layer 3 contains temporary changes of Layer 1 and 2. These changes are fixed for the whole duration of the scenario. They are not permanent in the sense of Layer 1 and 2.\n"\
"3) From Layer 3 upwards, state changes are introduced. Additionally, from Layer 4 upwards state changes can be time-dependent.\n"\
"4) If an entity has time-dependent properties (potentially variable during a scenario), it should be placed on Layer 4 upwards. However, not all its properties need to be time-dependent.\n"\
"5) Not all properties of an entity are necessarily in the same layer. The same property of a given entity should, however, not be located on different layers.\n"\
"6) Properties of all layers can influence properties on other layers. There is no single direction of influence.\n"

sys_prompt_segment__Output_format = "# Output: Your output should follow EXACTLY this format (MOST IMPORTANT):\n" \
"scene = {'L1':,'L2':,'L3':,'L4':,'L5':} \n" \
"To avoid errors, do not use the character ' inside the layer's description.\n"

sys_prompt_segment__Task ="# Task: Please only modify the layer specified in the prompt to generate an Edge Case and change nothing in the other layers (MOST IMPORTANT)\n"\
"Your output must contains EXACTLY THE SAME TEXT in every layer other than the one you are tasked to modify (MOST IMPORTANT)\n"\
"- Please note that the scenario you generate must not be identical or just a segment compared to the scenario mentioned in the above prompts! (IMPORTANT)\n"

sys_prompt_1 = sys_prompt_segment__Role + sys_prompt_segment__Layer_model + sys_prompt_segment__Output_format + sys_prompt_segment__Task
sys_prompt_2 = sys_prompt_segment__Role + sys_prompt_segment__Layer_model + sys_prompt_segment__Task


# Layer specific task prompts #######################

# the format is already specified in the Scenario Class
# structured 5LM prompts don't need to specify the output template, beyond explaining the 5LM 
# these prompts correspond to the prompt component that varies depending on which layer is being edited

Layer__specific_editing = []

# Layer 1
Layer__specific_editing.append(
    "Turn this scenario into an Edge Case by modifying only the layer L1 from the input.\n" \
    "You should prioritize creating rare or challenging road geometry or material, while making sure it remains physically possible.\n" \
    "You may also add rare, contradicting or complexe combinations of guidance objects\n" \
    "You do not need to edit every component of the layer, but should instead focus on making specific elements challenging.\n"
)

# Layer 2
Layer__specific_editing.append(

    "Turn this scenario into an Edge Case by modifying only the layer L2 from the input.\n" \
    "You should prioritize creating rare and challenging environments in way that may influence the scenario.\n" \
    "You may change the surrounding structures so as to change the geographical context of the scene.\n"
)

# Layer 3
Layer__specific_editing.append(

    "Turn this scenario into an Edge Case by modifying only the layer L3 from the input.\n" \
    "Try to come up with temprorary modification of L1 or L2 that could impact a vehicle's ability to navigate through the scene.\n"
)

# Layer 4
Layer__specific_editing.append(
    "Turn this scenario into an Edge Case by modifying only the layer L4 from the input.\n" \
    "You should either:\n" \
    "- Modify existing dynamic objects, or add new ones with rare and/or challenging characteristics. Look for object that do not belong in such a scenario.\n" \
    "- Modify the motion of existing dynamic objects, or add new objects with unique and challenging motion.\n" \
    "You may do both if needed, but focus on either the characteristics or the motion of the objects when generating a scenario.\n"
)

# Layer 5
Layer__specific_editing.append(
    "Turn this scenario into an Edge Case by modifying only the layer L5 from the input.\n" \
    "Take interaction between L5 and the other layers into account when adding weather condtions to make the scenario more challenging.\n" \
    "Think about unlikey and unexpected weather and/or illumination pattern for this scenario.\n" \
    "Pay attention to the environment parameter of L2 in particular. (Remember, DO NOT edit any layer other than L5)"
)


In [ ]:
#################################################################################################################################################################################################################################
# Utility functions
#################################################################################################################################################################################################################################

def make_prompt(task, input, expected_output):
    prompt = "# Task: \n"
    prompt += f"{task}\n"
    prompt += f"# Input: {input}\n"

    if expected_output != 1:
        prompt += f"- Please, generate {expected_output} new scenarios by repeating this task.\n"

    return prompt

def slicer(text, nb_scene, separator = ';;'):
    sliced_text = text
    sliced_scene_list = []

    for i in range(nb_scene-1):
        index = sliced_text.index(separator)
        sliced_scene = sliced_text[:index]
        sliced_text = sliced_text[index + len(separator):]
        sliced_scene_list.append(sliced_scene)
    sliced_scene_list.append(sliced_text)
    return(sliced_scene_list)

def filtering(s):
    string = s.replace('\',','",')
    string = string.replace(',\'',',"')
    string = string.replace(', \'',', "')

    string = string.replace('\':','":')
    string = string.replace(':\'',':"')

    string = string.replace(': \'',': "')
    string = string.replace('{\'','{"')
    string = string.replace('\'}','"}')

    return string

def str_to_dict(scenario_str_list):
    convert = []
    for s in scenario_str_list:
        dict = {}
        exec(f"scene = {s}",dict)
        convert.append(dict['scene'])
    return(convert)


# Create an empty response from gemini that can be used by the experiment function when there is an error (might need some rethinking of the code so that this isn't needed) 
response_null = client.models.generate_content(
        model= "gemini-2.5-flash",
        contents="reply with 'None'"
        )

#####################################################################################################################################################################
# Files management
#####################################################################################################################################################################

# Method to save the results of an experiment at a given path
# The method will create a new directory there to store the experiment files

def save_results(experiment_results, path, config : Config, json = False): 
    # Provide the base path to store experimental results. 
    # This will create a folder as 'experiment_X'. 
    # You may use the same base path for multiple experiments as this will avoid redundancy

    exp_iteration = 1
    exp_path = f'{path}/experiment_{exp_iteration}'
  
    while os.path.exists(exp_path):
        
        exp_iteration+=1
        exp_path = f'{path}/experiment_{exp_iteration}'
    print(exp_path)
    os.makedirs(exp_path)
    
    for i,ref in enumerate(experiment_results):

        for l,layer in enumerate(ref):

            for j,layer_it in enumerate(layer):

                if json:
                    scenario_txt = f"{layer_it}"
                else:
                    scenario_txt = f"{layer_it['scene']}"
                    
                txt_file_name = f'generated_scenario_r{i}_l{l}_it{j}.txt'
                txt_file_path = f"{exp_path}/{txt_file_name}"

                with open(txt_file_path, 'w') as file:
                    file.write(scenario_txt)

    # Specify any relevant information about the experiment in the config file 
    # You may change the Config class to specify additional experimental information to save
    config_file_name = f'config.txt'
    config_file_path = f"{exp_path}/{config_file_name}"

    with open(config_file_path, 'w') as file:
        print(config)
        file.write(config.__str__())

def open_file(PATH):
    f = open(PATH, 'r')
    lines = f.readlines()
    description = ""
    for l in lines:
        description += l
    return(description)

# Take the path to an experiment saved with save_results()
# Returns a list of all the saved scenarios as strings

def gather_strings(base_path):
    texts = []
    for file in os.listdir(base_path):
        if file != "config.txt":
            texts.append(open_file(base_path +"/"+ file))    
    return(texts)

# Take the path to an experiment saved with save_results() and a layer L
# Returns a list of all the scenarios where the layer L has been modified as strings

def gather_strings_per_layer(base_path, layer):
    texts = []
    for file in os.listdir(base_path):
        str = f'_l{layer-1}_'
        if file.find(str) > 0:
            texts.append(open_file(base_path +"/"+ file))
    return(texts)



def get_layer_str(path):
    layer_str = {}

    for i in range(NB_LAYER):
        L = f'L{i+1}'
        finder = []
        for s in gather_strings(path):
            dict = {}
            exec(f'scene = {s}', dict)
            layer = dict['scene'][L]
            finder.append(layer)
        layer_str[L] = finder
    return layer_str

def get_edited_layer_str(path, lay):
    layer_str = {}

    for i in range(NB_LAYER):
        L = f'L{i+1}'
        finder = []
        for s in gather_strings_per_layer(path,lay):
            dict = {}
            s = filtering(s)      # This method may be bugged for some generation results, will be fixed soon
            # try: 
            #     exec(f'scene = {s}', dict)
            # except:
            #     # s = filtering(s)
            print(s)
            dict = json.loads(s)
            layer = dict[L]
            finder.append(layer)
        layer_str[L] = finder
    return layer_str


#################################################################################################################################################################################################################################
# Main method for Scenario generation
#################################################################################################################################################################################################################################

# Input: a reference scenario, a layer L and a system prompt
# Output: 1 new scenario with layer L modified


def EC_generation(reference, edit_layer, sys_prompt, output_per_prompt = 1, model = "gemini-2.5-flash", output_structure = None, lvl = 1):
    input = ""
    task = f"- Please turn the input driving scenario into an Edge Case by changing the layer L{edit_layer}"

    if lvl == 2:
        task = Layer__specific_editing[edit_layer-1]  

    # Extract each layer from the reference and turn it into a string for the final prompt
    if isinstance(reference,type(list)):
        for i,L in enumerate(reference):
            input +=  f"L{i+1}:{L}"   
    elif isinstance(reference, type(dict)):
        for L in reference:
            input += f"{L}:{reference[L]}\n"

    prompt = make_prompt(task, input, output_per_prompt)

    try:
        if output_structure == None:
            response = client.models.generate_content(
            model= model,
            contents=prompt,
            config=GenerateContentConfig(
                system_instruction=[sys_prompt]
                ),
            )
        else:
            response = client.models.generate_content(
            model= model,
            contents= prompt,
            config= GenerateContentConfig(
                response_mime_type="application/json",
                response_schema = output_structure, 
                system_instruction= sys_prompt),
        )
    except:
        print("An error occured when calling the model")
        return response_null
    return response




#################################################################################################################################################################################################################################
# Main method for experimental set up
#################################################################################################################################################################################################################################

# Input: A list of reference scenes
#        The number of iteration per layer
#        Model and experiment parameters
# Output: generate multiple new scenes for every layer of every reference scene

def generation_experiment(base_scenes, layer_it, sys_prompt, retry_budget = 1, output_per_prompt = 1, show_details = False, model = "gemini-2.5-flash", output_structure = None, lvl =1):
    results = []
    failure_count = 0
    finale_failure = 0
  
    for s_ind,ref_scene in enumerate(base_scenes):          #generate new scenes from every reference scene
        scene_results=[]
        scene = json.loads(ref_scene)
        print(scene)
        for i in range(NB_LAYER):                           #generate new instance of every layers
            layer_results=[]
            for j in range(layer_it):                       #repeat layer_it times for each layer
                retry = 0
                finish_gen = False
                while not finish_gen and retry<=retry_budget:
                    LLM_generation = EC_generation(scene,i+1,sys_prompt, output_per_prompt=output_per_prompt, model = model, output_structure=output_structure, lvl=lvl)

                    # print(LLM_generation.text)
                    # print("--------------------")
                    #Attempting to read the results from the generation
                    try:
                        synthetic_scene = json.loads(LLM_generation.text)
                        if output_per_prompt !=1:
                            if len(synthetic_scene["output"])<output_per_prompt:
                                raise
                            else: 
                                for slice in synthetic_scene["output"]:
                                    if show_details:
                                        print(slice)
                                    layer_results.append(slice)
                                finish_gen = True
                                print(f'*** iteration succesful***')
                        else:
                            layer_results.append(synthetic_scene)
                            finish_gen = True
                            print(f'** iteration succesful**')


                        # synthetic_scene = {}
                        # if output_per_prompt !=1:
                        #     Generated_scenes = slicer(LLM_generation.text, output_per_prompt)
                        #     for slice in Generated_scenes:
                        #         if show_details:
                        #             print(slice)
                        #         synthetic_scene = {}
                        #         exec(slice, synthetic_scene)
                        #         layer_results.append(synthetic_scene)
                        #     finish_gen = True
                        #     if show_details:
                        #         print(f'*** iteration succesful***')
                        # else: 
                        #     exec(LLM_generation.text,synthetic_scene)
                        #     layer_results.append(synthetic_scene)
                        #     finish_gen = True
                        #     print(f'** iteration succesful**')

                    except:                        
                        failure_count+=1
                        if retry == retry_budget:
                            synthetic_scene = {}
                            print("Additional attempts failed\n******")
                            exec("scene = {'L1':'','L2':'','L3':'','L4':'','L5':''}",synthetic_scene)
                            layer_results.append(synthetic_scene)
                            finale_failure+=1
                            finish_gen = True
                        else:
                            print("Generation failure: synthetic scenario does not fit the template\nAttempting new generation...")
                            retry +=1
            scene_results.append(layer_results)
            print(f"Generation done for layer {i+1}...")
        results.append(scene_results)
        print(f"------------------------------------------\nGeneration done for scene {s_ind}\n------------------------------------------\n")
    print(f"total amount of failure: {failure_count}")
    print(f"failures in the final generation: {finale_failure}")

    config_file = Config(
        sys_prompt,
        layer_it,
        output_per_prompt,
        finale_failure,
        failure_count,
        model=model
    )

    return [results,config_file]


#################################################################################################################################################################################################################################
# Same as the previous functions, but generate 'rephrased' scenario that are as close as possible from the original scene. Change the words without changing the actual content. 
#################################################################################################################################################################################################################################


def Rephrase_generation(reference, edit_layer, sys_prompt, output_per_prompt = 1, model = "gemini-2.5-flash"):
    input = ""
    task = f"- Please, rephrase the layer L{edit_layer} completly without changing anything about the content of the scene." \
    "Be very careful that the scenario remains the same, but only described with different words (MOST IMPORTANT)"\
    "Only modify the specified layer, do not change the other layers (VERY IMPORTANT)"
    if isinstance(reference,list):
        for i,L in enumerate(reference):
            input +=  f"L{i+1}:{L}"   
    elif isinstance(reference, type(dict)):
        for L in reference:
            input += f"{L}:{reference[L]}"

    prompt = make_prompt(task, input, output_per_prompt)

    response = client.models.generate_content(
    model= model,
    contents=prompt,
    config=GenerateContentConfig(
        system_instruction=[sys_prompt]
        ),
    )
    return response

def generation_experiment_ref(base_scenes, layer_it, sys_prompt, retry_budget = 1, output_per_prompt = 1, show_details = False, model = "gemini-2.5-flash"):
    results = []
    failure_count = 0
    finale_failure = 0
  
    for s_ind,ref_scene in enumerate(base_scenes):          #generate new scenes from every reference scene
        scene_results=[]
        for i in range(NB_LAYER):                           #generate new instance of every layers
            layer_results=[]
            for j in range(layer_it):
                retry = 0
                finish_gen = False
                while not finish_gen and retry<=retry_budget:
                    LLM_generation = Rephrase_generation(ref_scene,i+1,sys_prompt, output_per_prompt=output_per_prompt, model = model)
                    try:
                        synthetic_scene = {}
                        if output_per_prompt !=1:
                            Generated_scenes = slicer(LLM_generation.text, output_per_prompt)
                            for slice in Generated_scenes:
                                if show_details:
                                    print(slice)
                                synthetic_scene = {}
                                exec(slice, synthetic_scene)
                                layer_results.append(synthetic_scene)
                            finish_gen = True
                            if show_details:
                                print(f'*** iteration succesful***')
                        else: 
                            exec(LLM_generation.text,synthetic_scene)
                            layer_results.append(synthetic_scene)
                            finish_gen = True
                            print(f'** iteration succesful**')

                    except:
                        
                        # print(LLM_generation.text)
                        failure_count+=1
                        if retry == retry_budget:
                            synthetic_scene = {}
                            print("Additional attempts failed\n******")
                            exec("scene = {'L1':'','L2':'','L3':'','L4':'','L5':'','L6':''}",synthetic_scene)
                            layer_results.append(synthetic_scene)
                            finale_failure+=1
                            finish_gen = True
                        else:
                            print("Generation failure: synthetic scenario does not fit the template\nAttempting new generation...")
                            retry +=1
            scene_results.append(layer_results)
            print(f"Generation done for layer {i+1}...")
        results.append(scene_results)
        print(f"------------------------------------------\nGeneration done for scene {s_ind}\n------------------------------------------\n")
    print("total amount of failure: " + str(failure_count))
    print("failures in the final generation: " + str(finale_failure))
    return results

# Evaluation metrics

In [ ]:
#################################################################################################################################################################################################################################
# 
#################################################################################################################################################################################################################################


def OOD_ness(texts_ref,texts, operation_type, model="gemini-embedding-001", show_details = False):
    embed_ref = [
        np.array(e.values) for e in client.models.embed_content(
            model = model,
            contents = texts_ref,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")).embeddings
            ]
    
    # Text embedding is done in chunks when there are too many texts, to avoid model saturation
    if len(texts)>100:
        partial_texts = texts[:100]
        ind = 0
        embeddings = []
        while len(partial_texts) > 0:
            embed_generation = [
                np.array(e.values) for e in client.models.embed_content(
                    model = model,
                    contents = partial_texts,
                    config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")).embeddings
                    ]
            if len(embeddings) == 0:
                embeddings = embed_generation
            else:
                embeddings = np.concatenate([embeddings, embed_generation])
            ind+=1
            partial_texts = texts[ind * 100 :(ind+1) * 100]
            print(len(partial_texts))
    else:
        embeddings = [
                np.array(e.values) for e in client.models.embed_content(
                    model = model,
                    contents = texts,
                    config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")).embeddings
                    ]
    

    embeddings_matrix_ref = np.array(embed_ref)
    embeddings_matrix_text = np.array(embeddings)
    embeddings_matrix = np.concatenate([embeddings_matrix_ref,embeddings_matrix_text])

    similarity_matrix = cosine_similarity(embeddings_matrix)

    average_sim = []
    for i, text in enumerate(texts):
        max_similarity= 0
        similarity_vect = []
        for j, _ in enumerate(texts_ref):
            similarity = similarity_matrix[i+len(texts_ref),j]
            similarity_vect.append(similarity)

        if operation_type == 'min':
            O_i = np.min(similarity_vect)
        elif operation_type == 'max':
            O_i = np.max(similarity_vect)
        else:
            return "Wrong operation type. Choose \'min\' or \'max\'."
        average_sim.append(O_i)
        if(show_details):
            print(f"max distance between reference data and {text} is {max_similarity}")

    return np.average(average_sim)


#################################################################################################################################################################################################################################
# 
#################################################################################################################################################################################################################################


def diversity(texts, operation_type, model = "gemini-embedding-001", show_details = False, show_matrix = False):

    # Text embedding is done in chunks when there are too many texts, to avoid model saturation
    if len(texts)>100:
        partial_texts = texts[:100]
        ind = 0
        embeddings = []
        while len(partial_texts) > 0:
            embed_generation = [
                np.array(e.values) for e in client.models.embed_content(
                    model = model,
                    contents = partial_texts,
                    config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")).embeddings
                    ]
            if len(embeddings) == 0:
                embeddings = embed_generation
            else:
                embeddings = np.concatenate([embeddings, embed_generation])
            ind+=1
            partial_texts = texts[ind * 100 :(ind+1) * 100]
    else:
        embeddings = [
                np.array(e.values) for e in client.models.embed_content(
                    model = model,
                    contents = texts,
                    config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")).embeddings
                    ]

    embeddings_matrix = np.array(embeddings)
    similarity_matrix = cosine_similarity(np.vstack([embeddings_matrix]))

    diversity_vect = []
    for i, text1 in enumerate(texts):
        similarity_vect = []
        for j in range(i + 1, len(texts)):
            text2 = texts[j]
            similarity = similarity_matrix[i, j]
            similarity_vect.append(similarity)
            if show_details:
                print(f"Similarity between '{text1}' and '{text2}': {similarity:.4f}")
        if len(similarity_vect)>0:

            if operation_type == 'min':
                D_i = np.min(similarity_vect)
            elif operation_type == 'max':
                D_i = np.max(similarity_vect)
            elif operation_type == 'avg':
                D_i = np.average(similarity_vect)
            else:
                return "Wrong operation type. Choose \'min\', \'max\' or \'avg\'."
            diversity_vect.append(D_i)

            if show_details:
                print(f"{operation_type} similarity of '{i}' is: {D_i:.4f}")

    diversity = np.average(diversity_vect)
    if show_details:
        print(f"diversity = {diversity:.4f} ")
    if show_matrix:
        sns.heatmap(similarity_matrix, vmin=0, vmax=1, cmap="Greens")
    return diversity


#################################################################################################################################################################################################################################
# Metrics applied to layers individually
#################################################################################################################################################################################################################################


def layer_diversity(path, layer):

    experiment_str = get_layer_str(path)

    diversities_min = []
    diversities_max = []
    diversities_avg = []

    filtered_str = []
    for s in experiment_str[layer]:
        if len(s) != 0:
            filtered_str.append(str(s))

    diversities_min.append(diversity(filtered_str, 'min'))
    diversities_avg.append(diversity(filtered_str, 'avg'))
    diversities_max.append(diversity(filtered_str, 'max'))

    return([diversities_min,diversities_avg,diversities_max])

def layer_ood(path, layer, ref):
    L = f'L{layer}'
    experiment_str = get_edited_layer_str(path, layer)

    filtered_str = []
    for s in experiment_str[L]:
        if len(s) != 0:
            filtered_str.append(str(s))

    print(f"Number of evaluated samples: {len(filtered_str)}")
    OOD_max = OOD_ness(ref, filtered_str, 'max')
    OOD_min = OOD_ness(ref, filtered_str, 'min')

    return([OOD_min,OOD_max])

#################################################################################################################################################################################################################################
# Metrics applied to specific components of layers individually
#################################################################################################################################################################################################################################

def component_diversity(path, layer):
    experiment_str = get_edited_layer_str(path, int(layer[-1]))
    filtered_str = []
    for s in experiment_str[layer]:
        if len(s) != 0:
            filtered_str.append(s)

    print(f'edited scenes: {len(filtered_str)}')

    if layer == 'L1':
        road = []
        guidance = []
        for s in filtered_str:
            road.append(str(s['roads']))
            guidance.append(str(s['guidance_objects']))
        print(f'samples r: {len(road)}')
        print(f'samples g: {len(guidance)}')

        print("min diversity of input roads in layers L1:" )
        print(diversity(road, 'max'))
        print("min diversity of input guidance objects in layers L1:" )
        print(diversity(guidance, 'max'))
        
    elif layer == 'L2': 
        environment = []
        structures = []
        for s in filtered_str:
            environment.append(str(s['environment']))
            structures.append(str(s['structures']))
        print(f'samples e: {len(environment)}')
        print(f'samples s: {len(structures)}')

        print("min diversity of input environment in layers L2:" )
        print(diversity(environment, 'max'))
        print("min diversity of input structures in layers L2:" )
        print(diversity(structures, 'max'))

    elif layer == 'L3': 
        characteristics = []
        for s in filtered_str:
            print(len(s['change_list']))
            for obj in s['change_list']:
                
                characteristics.append(str(obj['characteristic']))
        print(f'samples c: {len(characteristics)}')

        print("min diversity of input characteristic in layers L4:" )
        print(diversity(characteristics, 'max'))

    elif layer == 'L4': 
        type = []
        characteristics = []
        motion = []
        for s in filtered_str:
            print(len(s['object_list']))
            for obj in s['object_list']:
                
                type.append(str(obj['type']))
                characteristics.append(str(obj['characteristic']))
                motion.append(str(obj['motion']))
        print(f'samples t: {len(type)}')
        print(f'samples c: {len(characteristics)}')
        print(f'samples m: {len(motion)}')

        print("min diversity of input type in layers L4:" )
        print(diversity(type, 'max'))
        print("min diversity of input characteristic in layers L4:" )
        print(diversity(characteristics, 'max'))
        print("min diversity of input motion in layers L4:" )
        print(diversity(motion, 'max'))
        

    elif layer == 'L5': 
        weather = []
        illumination = []
        for s in filtered_str:
            weather.append(str(s['weather']))
            illumination.append(str(s['illumination']))
        print(f'samples w: {len(weather)}')
        print(f'samples i: {len(illumination)}')

        print("min diversity of input weather in layers L5:" )
        print(diversity(weather, 'max'))
        print("min diversity of input illumination in layers L5:" )
        print(diversity(illumination, 'max'))

#################################################################################################################################################################################################################################
# Quantitative evaluation function to display specific parts of the given scenarios and compare them more easily
#################################################################################################################################################################################################################################
def compare_scenarios(scenario1, scenario2):
    for l in range(NB_LAYER):
        L = f'L{l+1}'
        print(scenario1[L])
        print("---")
        print(scenario2[L])
        print("---")
        print(scenario1[L]==scenario2[L])
        print("------------------------------\n------------------------------")

def compare_saved_scenarios(path1, path2):
    f1 = open(path1, 'r')
    f2 = open(path2, 'r')
    lines_1 = f1.readlines()
    lines_2 = f2.readlines()
    scenario1 = ""
    scenario2 = ""
    d1 = {}
    d2 = {}

    for line in lines_1:
        scenario1 += line
    exec(f'scene = {scenario1}', d1)
    scenario1 = d1['scene']

    for line in lines_2:
        scenario2 += line
    exec(f'scene = {scenario2}', d2)
    scenario2 = d2['scene']

    compare_scenarios(scenario1,scenario2)


# Method to verify if a given generated scene only modified 1 layer without changing the other, or if there are modifications in the other layers as well.

def conformity(ref, generated):
    non_conformity = 0
    for i in range(NB_LAYER):
        L = f'L{i+1}'
        if ref[L] != generated[L]:
            non_conformity +=1
    return(non_conformity<2)

def conformity_check(ref, path):
    f = open(path, 'r')
    lines = f.readlines()
    d = {}
    scenario = ''
    for line in lines:
        scenario += line
    exec(f'scene = {scenario}', d)
    scenario = d['scene']
    return(conformity(ref,scenario))

In [ ]:
def type_embedding(component_list, enum_class):
    """ 
    Take a component list corresponding to a scene, and the expected enum for the class of object type.

    return the embedding vector of that scene within that class's type space.
    """
    embedding_vector = {}

    for i,type in enumerate(enum_class):
        embedding_vector[type.value] = 0

    # Hotfix to solve the inconsistancy in names for the Type parameter of the objects 
    if enum_class == Light_source:
        for component in component_list:
            print(component_list)
            embedding_vector[component['light_source']]+=1
    else:
        for component in component_list:
            try:
                embedding_vector[component['Type']]+=1
            except:
                embedding_vector[component['type']]+=1
    return embedding_vector



def custom_diversity(input_layers, list_name, enum_class, show_matrix = False):
    embedding_matrix = []
    for layer in input_layers:
        components = layer[list_name]
        embed = type_embedding(components, enum_class)
        embedding_list = [item[1] for item in embed.items()]
        
        embedding_matrix.append(np.array([embedding_list]))

    matrix = np.concatenate(embedding_matrix, axis=0) 
    if show_matrix:
        print(matrix)
    sim_matrix = cosine_similarity(matrix)


    
    values_list = []
    for i in range(len(input_layers)):

        simi_list = []
        for j, _ in enumerate(input_layers):
            simi_list.append(sim_matrix[i,j])

        simi_list.pop(i)
        values_list.append([
            min(simi_list),
            max(simi_list),
            sum(simi_list)/len(simi_list)
        ])
        print('----------')
        print(f"max distance between reference data and {i} is {values_list[-1][0]}")
        print(f"min distance between reference data and {i} is {values_list[-1][1]}")
        print(f"average distance between reference data and {i} is {values_list[-1][2]}")

    average_values = [
        sum([values[0] for values in values_list])/len([values[0] for values in values_list]),
        sum([values[1] for values in values_list])/len([values[1] for values in values_list]),
        sum([values[2] for values in values_list])/len([values[2] for values in values_list])

    ]
    print('----------------\nAverage values:\n')
    print(f"max distance : {average_values[0]}")
    print(f"min distance : {average_values[1]}")
    print(f"average distance : {average_values[2]}")

# structured_ref_scene_layers4 = [json.loads(ref)['L4'] for ref in structured_ref_scene]
# custom_diversity(structured_ref_scene_layers4, 'object_list', Dynamic_object_type)

def custom_ood(input_layers, list_name, enum_class, ref_layers):
    embedding_matrix = []

    for layer in ref_layers:
        components_ref = layer[list_name]
        embed_ref = type_embedding(components_ref, enum_class)
        embedding_list_ref = [item[1] for item in embed_ref.items()]
        
        embedding_matrix.append(np.array([embedding_list_ref])) 
    
    for layer in input_layers:
        components = layer[list_name]
        embed = type_embedding(components, enum_class)
        embedding_list = [item[1] for item in embed.items()]
        
        embedding_matrix.append(np.array([embedding_list]))

    matrix = np.concatenate(embedding_matrix, axis=0) 
    print(matrix)
    sim_matrix = cosine_similarity(matrix)


    values_list = []
    for i in range(len(input_layers)):
        simi_list = []
        for j in range(len(ref_layers)):
            simi_list.append(sim_matrix[i+len(ref_layers),j])

        # simi_list.pop(i)
        values_list.append([
            min(simi_list),
            max(simi_list),
            sum(simi_list)/len(simi_list)
        ])
        print('----------')
        print(f"max distance between reference data and {i} is {values_list[-1][0]}")
        print(f"min distance between reference data and {i} is {values_list[-1][1]}")
        print(f"average distance between reference data and {i} is {values_list[-1][2]}")

    average_values = [
        sum([values[0] for values in values_list])/len([values[0] for values in values_list]),
        sum([values[1] for values in values_list])/len([values[1] for values in values_list]),
        sum([values[2] for values in values_list])/len([values[2] for values in values_list])

    ]
    print('----------------\nAverage values:\n')
    print(f"max distance : {average_values[0]}")
    print(f"min distance : {average_values[1]}")
    print(f"average distance : {average_values[2]}")



def custom_component_metrics(path, layer, metric='div', reference = None):
    experiment_str = get_edited_layer_str(path, int(layer[-1]))
    # experiment_str = [json.loads(scene) for scene in gather_strings_per_layer(path, int(layer[-1]) )]
    filtered_str = []
    for s in experiment_str[layer]:
        if len(s) != 0:
            filtered_str.append(s)

    print(f'edited scenes: {len(filtered_str)}')

    if metric == 'ood':
        structured_ref_scene_layer = [json.loads(ref)[layer] for ref in reference]


    if layer == 'L1':
        road = []
        guidance = []
        for s in filtered_str:
            road.append(str(s['roads']))
            guidance.append(str(s['guidance_objects']))
        print(f'samples r: {len(road)}')
        print(f'samples g: {len(guidance)}')

        if metric == 'ood':
            custom_ood(filtered_str, 'roads', Road_type, structured_ref_scene_layer)
        else:
            custom_diversity(filtered_str, 'roads', Road_type)

        # print("min diversity of input roads in layers L1:" )
        # print(diversity_max(road, 'min'))
        # print("min diversity of input guidance objects in layers L1:" )
        # print(diversity_max(guidance, 'min'))
        
    elif layer == 'L2': 
        environment = []
        structures = []
        for s in filtered_str:
            environment.append(str(s['environment']))
            structures.append(str(s['structures']))
        print(f'samples e: {len(environment)}')
        print(f'samples s: {len(structures)}')

        if metric == 'ood':
            custom_ood(filtered_str, 'structures', Structure_type, structured_ref_scene_layer)
        else:
            custom_diversity(filtered_str, 'structures', Structure_type)


        # print("min diversity of input environment in layers L2:" )
        # print(diversity(environment, 'min'))
        # print("min diversity of input structures in layers L2:" )
        # print(diversity(structures, 'min'))

    elif layer == 'L3': 
        pass
    elif layer == 'L4': 
        type = []
        characteristics = []
        motion = []
        nb_object = []
        for s in filtered_str:
            nb_object.append(len(s['object_list']))
            # print(nb_object[-1])
            for obj in s['object_list']:

                print(obj)
                type.append(str(obj['type']))
                characteristics.append(str(obj['characteristic']))
                motion.append(str(obj['motion_type']))
        print(f'average objects per scene :{np.average(nb_object)}')
        print(f'samples t: {len(type)}')
        print(f'samples c: {len(characteristics)}')
        print(f'samples m: {len(motion)}')

        
        if metric == 'ood':
            custom_ood(filtered_str, 'object_list', Dynamic_object_type, structured_ref_scene_layer)
        else:
            custom_diversity(filtered_str, 'object_list', Dynamic_object_type)


        # print("min diversity of input type in layers L4:" )
        # print(diversity(type, 'min'))
        # print("min diversity of input characteristic in layers L4:" )
        # print(diversity(characteristics, 'min'))
        # print("min diversity of input motion in layers L4:" )
        # print(diversity(motion, 'min'))
        

    elif layer == 'L5': 
        weather = []
        illumination = []
        for s in filtered_str:
            weather.append(str(s['weather']))
            illumination.append(str(s['illumination']))
        print(f'samples w: {len(weather)}')
        print(f'samples i: {len(illumination)}')

        if metric == 'ood':
            custom_ood(filtered_str, 'illumination', Light_source, structured_ref_scene_layer)
        else:
            custom_diversity(filtered_str, 'illumination', Light_source)


        # print("min diversity of input weather in layers L5:" )
        # print(diversity(weather, 'min'))
        # print("min diversity of input illumination in layers L5:" )
        # print(diversity(illumination, 'min'))

        

In [ ]:
# Example of structured scenario descriptions generated from the 10 scenes of ther nuScenes mini dataset
# These are used as the reference for both the generation and the evaluation metrics
structured_ref_scene_2 = ['{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 4,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      },\n      {\n        "type": "bus_lane",\n        "geometry": "straight",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 1,\n        "lane_size": 3.0,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      },\n      {\n        "type": "parking",\n        "geometry": "straight",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 2.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "Double yellow solid lines",\n        "position": "center of the road"\n      },\n      {\n        "type": "marking",\n        "characteristics": "White solid line",\n        "position": "right edge of the road"\n      },\n      {\n        "type": "marking",\n        "characteristics": "White crosswalk lines",\n        "position": "across the road at the initial intersection"\n      },\n      {\n        "type": "marking",\n        "characteristics": "White text \'BUS\'",\n        "position": "on the surface of the bus lane"\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Vertical traffic light assembly, showing yellow",\n        "position": "overhead at the intersection"\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Vertical traffic light assembly for pedestrians",\n        "position": "on a pole on the left sidewalk"\n      },\n      {\n        "type": "sign",\n        "characteristics": "One Way street sign",\n        "position": "on a pole on the right side of the intersection"\n      },\n      {\n        "type": "sign",\n        "characteristics": "Street sign \'SLEEPER ST\'",\n        "position": "on a pole on the right sidewalk"\n      },\n      {\n        "type": "sign",\n        "characteristics": "Yellow pedestrian crossing warning sign",\n        "position": "on a pole on the right sidewalk"\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "Large, multi-story brick buildings with numerous windows.",\n        "position": "lining both sides of the street"\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "Deciduous trees with green leaves.",\n        "position": "planted along the sidewalks on both sides of the street"\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Concrete sidewalks with curbs.",\n        "position": "along both sides of the street"\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Ornate black metal streetlamps.",\n        "position": "at regular intervals along both sidewalks"\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Metal bike racks.",\n        "position": "on the left sidewalk"\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "obstacle",\n        "characteristic": "Orange traffic cone",\n        "position": "on the right sidewalk"\n      },\n      {\n        "type": "obstacle",\n        "characteristic": "Orange traffic cone",\n        "position": "on the road next to a parked work van on the left"\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Black SUV",\n        "position": "in front of the ego vehicle, turning left",\n        "motion_type": "moving erratically",\n        "motion_characteristic": "turning left across the ego vehicle\'s path"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Black SUV",\n        "position": "pulling out from parking on the left and proceeding in front of the ego vehicle",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "accelerating after pulling into the lane"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Silver sedan",\n        "position": "pulling out from parking on the left and proceeding ahead of the black SUV",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "accelerating after pulling into the lane"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple cars and SUVs of various colors",\n        "position": "parked along both sides of the road",\n        "motion_type": "not moving",\n        "motion_characteristic": "stationary"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple cars and SUVs",\n        "position": "far ahead of the ego vehicle in the same lane",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "driving at a steady speed"\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Multiple people walking",\n        "position": "on the sidewalks on both sides of the road",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "walking along the sidewalk"\n      },\n      {\n        "type": "cyclist",\n        "characteristic": "Person riding a bicycle",\n        "position": "on the left side of the road, near the parked cars",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "cycling at a steady speed"\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "cloudy",\n      "characteristics": "The sky is overcast with grey clouds, but the road is dry."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Bright, diffuse daylight due to cloud cover."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 4,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "wet surface"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "traffic light",\n        "characteristics": "The traffic light for the ego vehicle is green at the beginning of the video.",\n        "position": "On a pole to the left of the intersection."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White dashed lines separating lanes.",\n        "position": "On the road surface along the path of the ego vehicle."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White pedestrian crossing lines (zebra crossing).",\n        "position": "Across the road at the initial intersection."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White text \'SLOW\' painted on the road surface.",\n        "position": "In the ego vehicle\'s lane towards the end of the video."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White directional arrows.",\n        "position": "Painted on the road surface."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Blue rectangular directional sign.",\n        "position": "On the left side of the road."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Red and white triangular warning sign with \'SLOW\' text below it.",\n        "position": "On a pole on the left side of the road."\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "A large, illuminated commercial building named \'Holland Road Shopping Centre\'.",\n        "position": "Across the intersection at the start of the video."\n      },\n      {\n        "type": "skyscraper",\n        "characteristics": "Tall, illuminated residential buildings.",\n        "position": "In the distant background."\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "Trees and bushes in planters and median strips.",\n        "position": "Along both sides of the road."\n      },\n      {\n        "type": "wall or barrier",\n        "characteristics": "Metal railings separating traffic from the sidewalk and opposing lanes.",\n        "position": "Along the roadside and in the center median."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Tall metal streetlight poles.",\n        "position": "Periodically along both sides of the road."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Covered bus stop shelters.",\n        "position": "On the sidewalk to the left."\n      }\n    ]\n  },\n  "L3": {\n    "change_list": []\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Silver Toyota Previa minivan.",\n        "position": "Directly in front of the ego vehicle.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Accelerating away from the intersection and driving straight ahead."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Multiple pedestrians, some with an umbrella.",\n        "position": "Crossing the road in the pedestrian crossing at the start of the video.",\n        "motion_type": "moving toward ego",\n        "motion_characteristic": "Walking across the road from both left to right and right to left."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White van and a white truck.",\n        "position": "To the left of the ego vehicle at the intersection.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Moving through the intersection."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple cars and a bus.",\n        "position": "In the lanes to the right and in the opposing traffic lanes.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving straight in their respective lanes."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Multiple people standing on the sidewalk.",\n        "position": "On the left sidewalk, some near a bus stop.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Waiting or standing still."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "The road surface is wet and reflective, indicating recent rainfall, but there is no active precipitation."\n    },\n    "illumination": [\n      {\n        "light_source": "lamps",\n        "characteristic": "Bright white light from numerous streetlights lining the road."\n      },\n      {\n        "light_source": "vehicle_light",\n        "characteristic": "Red taillights and white headlights from all vehicles on the road."\n      },\n      {\n        "light_source": "neon",\n        "characteristic": "Illumination from shop windows, signs, and advertisements on the buildings."\n      },\n      {\n        "light_source": "reflection",\n        "characteristic": "Strong reflections from all light sources are visible on the wet asphalt."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "minor cracks and patches"\n      },\n      {\n        "type": "parking",\n        "geometry": "straight",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 2.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      },\n      {\n        "type": "bus_lane",\n        "geometry": "straight",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 1,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "Double solid yellow line",\n        "position": "center of the road"\n      },\n      {\n        "type": "marking",\n        "characteristics": "Solid white line",\n        "position": "right edge of the road"\n      },\n      {\n        "type": "marking",\n        "characteristics": "Dashed white line",\n        "position": "between the main lane and the bus lane on the right"\n      },\n      {\n        "type": "marking",\n        "characteristics": "Crosswalk with white stripes",\n        "position": "across the road at intersections"\n      },\n      {\n        "type": "marking",\n        "characteristics": "White arrow pointing straight",\n        "position": "on the road surface in the ego-vehicle\'s lane"\n      },\n      {\n        "type": "sign",\n        "characteristics": "No Parking sign",\n        "position": "on the right sidewalk"\n      },\n      {\n        "type": "sign",\n        "characteristics": "Bus stop sign",\n        "position": "on the right sidewalk"\n      },\n      {\n        "type": "sign",\n        "characteristics": "Parking information sign",\n        "position": "on the sidewalk"\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Standard traffic light",\n        "position": "on the left side of the road, further ahead"\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "Large, modern office or commercial buildings with many windows.",\n        "position": "lining both sides of the street"\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "Deciduous trees of medium height.",\n        "position": "planted along the sidewalks on both sides of the road"\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Metal street light poles.",\n        "position": "along the sidewalks on both sides"\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Wooden utility poles with overhead wires.",\n        "position": "on the left side of the road, further down"\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "road work",\n        "characteristic": "Utility work being performed, indicated by the presence of utility trucks with boom lifts.",\n        "position": "on the left side of the road, occupying the parking lane"\n      },\n      {\n        "type": "static obstacle",\n        "characteristic": "Orange traffic cones.",\n        "position": "on the left side of the road, delineating a work area"\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Various parked cars including sedans, SUVs, and vans.",\n        "position": "in designated parking spots along both sides of the road",\n        "motion_type": "not moving",\n        "motion_characteristic": "parked"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White utility truck with a boom lift.",\n        "position": "parked on the left side of the road in a work zone",\n        "motion_type": "not moving",\n        "motion_characteristic": "parked for road work"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Black sedan.",\n        "position": "initially parked on the left, pulls out and drives ahead of the ego vehicle",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "accelerating to match traffic speed"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Articulated city bus.",\n        "position": "stopped in the bus lane on the right side of the road",\n        "motion_type": "not moving",\n        "motion_characteristic": "stopped at a bus stop"\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Person in casual clothing.",\n        "position": "crossing the road in the distance from left to right",\n        "motion_type": "other",\n        "motion_characteristic": "walking across the street"\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Person in casual clothing.",\n        "position": "crossing the road on a crosswalk ahead of the ego vehicle, from left to right",\n        "motion_type": "other",\n        "motion_characteristic": "walking across the street"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Black SUV.",\n        "position": "on the left side of the road, pulls into the ego-vehicle\'s lane and drives ahead",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "changing lanes and accelerating"\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "Sunny day with a few scattered clouds. The road surface is dry."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Bright, direct sunlight causing strong, distinct shadows from buildings and trees."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "road",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 6,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      },\n      {\n        "type": "road",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 4,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "traffic light",\n        "characteristics": "Red light for the ego vehicle\'s direction of travel.",\n        "position": "Suspended over the center of the intersection."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White painted crosswalk markings.",\n        "position": "Directly in front of the ego vehicle across the intersection."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White painted straight arrow and bicycle symbol.",\n        "position": "In the ego vehicle\'s lane."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Street sign that reads \'SUMMER ST\'.",\n        "position": "Suspended over the intersection."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Regulatory sign that reads \'NO TURN ON RED\'.",\n        "position": "On a pole on the right side of the intersection."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Highway shield for Interstate 90.",\n        "position": "On a pole above the traffic lights."\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "A large, modern convention center or similar building with glass and panel facade.",\n        "position": "On the left side of the road."\n      },\n      {\n        "type": "multy-story building",\n        "characteristics": "A multi-level parking garage or office building.",\n        "position": "On the right side of the road."\n      },\n      {\n        "type": "skyscraper",\n        "characteristics": "Several tall city buildings forming the skyline.",\n        "position": "In the distant background."\n      },\n      {\n        "type": "bridge",\n        "characteristics": "A large overpass or bridge structure for a highway.",\n        "position": "Crossing over the road far ahead."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Multiple tall construction cranes.",\n        "position": "In the background, behind the buildings."\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "A line of deciduous trees.",\n        "position": "Along the sidewalk on the left."\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "Construction barriers",\n        "characteristic": "Orange and white plastic temporary barriers.",\n        "position": "In the background, sectioning off a construction area."\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "cyclist",\n        "characteristic": "A person riding a bicycle.",\n        "position": "Crossing the intersection from right to left in the distance at the start of the video.",\n        "motion_type": "other",\n        "motion_characteristic": "Moving at a steady pace perpendicular to the ego vehicle."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple sedans and SUVs.",\n        "position": "Crossing the intersection in front of the ego vehicle from left to right.",\n        "motion_type": "other",\n        "motion_characteristic": "Moving at a steady speed with the flow of traffic."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "A dark-colored pickup truck.",\n        "position": "Crossing the intersection in front of the ego vehicle from left to right.",\n        "motion_type": "other",\n        "motion_characteristic": "Moving at a steady speed with the flow of traffic."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "A white shuttle bus or paratransit van.",\n        "position": "Crossing the intersection in front of the ego vehicle from left to right.",\n        "motion_type": "other",\n        "motion_characteristic": "Moving at a steady speed with the flow of traffic."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "A large group of people of various ages.",\n        "position": "Crossing the road from left to right on the crosswalk directly in front of the ego vehicle.",\n        "motion_type": "other",\n        "motion_characteristic": "Walking at a normal pace, perpendicular to the ego vehicle."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "An elderly person with a red shopping trolley.",\n        "position": "Crossing the road from left to right on the crosswalk directly in front of the ego vehicle.",\n        "motion_type": "other",\n        "motion_characteristic": "Walking slowly, perpendicular to the ego vehicle."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "The sky is bright and slightly hazy, with no visible clouds."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Bright, direct daylight casting long, dark shadows on the ground."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none visible"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "White painted stop line and crosswalk markings.",\n        "position": "on the road surface at the intersection ahead"\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Traffic light showing a red signal for the ego vehicle\'s direction.",\n        "position": "on a pole to the right of the intersection"\n      },\n      {\n        "type": "sign",\n        "characteristics": "Street name sign \'BOLTON ST\' and a lane direction sign (straight or right turn).",\n        "position": "on a pole on the left sidewalk"\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Multiple traffic lights for cross-traffic.",\n        "position": "at various corners of the intersection"\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "A large brick building with grey accents.",\n        "position": "on the left side of the road"\n      },\n      {\n        "type": "multy-story building",\n        "characteristics": "A building under construction, covered in black netting and scaffolding.",\n        "position": "on the left side of the road, at the corner of the intersection"\n      },\n      {\n        "type": "multy-story building",\n        "characteristics": "A modern, multi-story brick building with large windows.",\n        "position": "on the right side of the road, past the intersection"\n      },\n      {\n        "type": "wall or barrier",\n        "characteristics": "A chain-link fence enclosing a construction site.",\n        "position": "along the immediate right side of the road"\n      },\n      {\n        "type": "skyscraper",\n        "characteristics": "Tall city buildings are visible in the distance.",\n        "position": "in the background, straight ahead"\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "Construction site",\n        "characteristic": "A fenced-off construction area.",\n        "position": "on the right side of the road"\n      },\n      {\n        "type": "Obstacle",\n        "characteristic": "An orange and white striped traffic barrel.",\n        "position": "on the right edge of the road, before the intersection"\n      },\n      {\n        "type": "Obstacle",\n        "characteristic": "A red cherry picker lift vehicle parked.",\n        "position": "on the right, inside the construction fence"\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Black SUV",\n        "position": "crossing the intersection from left to right",\n        "motion_type": "other",\n        "motion_characteristic": "moving at a constant speed across the ego vehicle\'s path"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White public transit bus",\n        "position": "crossing the intersection from right to left",\n        "motion_type": "other",\n        "motion_characteristic": "moving at a constant speed across the ego vehicle\'s path"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple sedans of various colors (silver, black, white, green)",\n        "position": "crossing the intersection from both left and right",\n        "motion_type": "other",\n        "motion_characteristic": "moving at a constant speed across the ego vehicle\'s path"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Red \'Coca-Cola\' branded van",\n        "position": "crossing the intersection from left to right",\n        "motion_type": "other",\n        "motion_characteristic": "moving at a constant speed across the ego vehicle\'s path"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White and black dump truck",\n        "position": "crossing the intersection from left to right",\n        "motion_type": "other",\n        "motion_characteristic": "moving at a constant speed across the ego vehicle\'s path"\n      },\n      {\n        "type": "cyclist",\n        "characteristic": "Person riding a bicycle",\n        "position": "crossing the intersection from right to left in the distance",\n        "motion_type": "other",\n        "motion_characteristic": "pedaling at a steady speed"\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White \'AETNA\' branded van",\n        "position": "crossing the intersection from left to right",\n        "motion_type": "other",\n        "motion_characteristic": "moving at a constant speed across the ego vehicle\'s path"\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Several pedestrians",\n        "position": "on the sidewalk to the left",\n        "motion_type": "other",\n        "motion_characteristic": "walking along the sidewalk"\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "cloudy",\n      "characteristics": "The sky is overcast and the road surface is dry."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Natural, diffuse daylight due to heavy cloud cover."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 5,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      },\n      {\n        "type": "road",\n        "geometry": "turning left",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 3.0,\n        "surface_material": "asphalt",\n        "eventual_damages": "none"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "White dashed lines separating lanes.",\n        "position": "On the road surface along the ego vehicle\'s path."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White painted arrow pointing forward on the road surface.",\n        "position": "In the center of the ego vehicle\'s lane initially."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White painted arrow indicating a left turn.",\n        "position": "In the center of the lane before the intersection."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White rectangular blocks forming a crosswalk.",\n        "position": "Across the road at the intersection."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Triangular \'GIVE WAY\' sign.",\n        "position": "On a pole on the right side of the road before the intersection."\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Green light indicating permission to proceed.",\n        "position": "Suspended over the intersection ahead."\n      },\n      {\n        "type": "marking",\n        "characteristics": "Double yellow lines on the side of the road.",\n        "position": "On the left side of the road after the turn."\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "Large, modern building with glass and concrete facade.",\n        "position": "On the left side of the road."\n      },\n      {\n        "type": "multy-story building",\n        "characteristics": "Multi-story building with green facade elements.",\n        "position": "On the right side of the road in the distance."\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "Numerous trees and shrubs.",\n        "position": "Lining both sides of the road."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Streetlight poles.",\n        "position": "Along the sidewalks on both sides of the road."\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "Static Obstacle",\n        "characteristic": "Large delivery truck parked for unloading, partially blocking the leftmost lane.",\n        "position": "On the left side of the road at the beginning of the clip."\n      },\n      {\n        "type": "Roadwork",\n        "characteristic": "Red and white plastic temporary barriers sectioning off the right side of the road.",\n        "position": "On the right side of the road."\n      },\n      {\n        "type": "Roadwork",\n        "characteristic": "Construction site containing an orange excavator.",\n        "position": "On the left side of the intersection."\n      },\n      {\n        "type": "Roadwork",\n        "characteristic": "Orange and white traffic cones.",\n        "position": "Lining the right side of the road after the left turn."\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Large white delivery truck with \'POKKA LOGISTICS\' branding.",\n        "position": "Parked on the left side of the road.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Stationary for unloading."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "A man in a red polo shirt and beige pants.",\n        "position": "Next to the delivery truck on the left.",\n        "motion_type": "other",\n        "motion_characteristic": "Organizing cargo on the truck."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White minivan (Toyota Alphard).",\n        "position": "In front of the ego vehicle.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving at normal speed, then turning left at the intersection."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Black SUV.",\n        "position": "In the lane to the right of the ego vehicle.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving at normal speed."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Grey pickup truck.",\n        "position": "In the distance, in the same direction of travel.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving as part of traffic flow."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "A group of multiple people.",\n        "position": "On the right sidewalk, then crossing the street.",\n        "motion_type": "other",\n        "motion_characteristic": "Walking across the crosswalk from right to left."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Construction workers in high-visibility vests and hard hats.",\n        "position": "On the sidewalk near the intersection on the left.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Standing at the worksite."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Orange excavator.",\n        "position": "Inside the construction zone on the left.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Stationary."\n      },\n      {\n        "type": "cyclist",\n        "characteristic": "A parked bicycle.",\n        "position": "On the sidewalk to the left, after the turn.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Parked and leaning against a signpost."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "The road is dry and the sky is clear with a few scattered clouds."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Bright daylight causing distinct shadows from buildings and trees."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 4,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "Patched asphalt in the ego lane."\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "Double solid yellow lines indicating the edge of the carriageway.",\n        "position": "Along the left edge of the road."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White dashed lines separating lanes travelling in the same direction.",\n        "position": "Between the lanes."\n      },\n      {\n        "type": "marking",\n        "characteristics": "Yellow criss-cross box junction markings.",\n        "position": "On the road surface at an upcoming intersection."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White arrow pointing straight ahead painted on the road.",\n        "position": "In the middle of the ego vehicle\'s lane."\n      },\n      {\n        "type": "sign",\n        "characteristics": "A circular \'No U-turn\' sign.",\n        "position": "On a pole on the left side of the road."\n      },\n      {\n        "type": "sign",\n        "characteristics": "An orange, diamond-shaped temporary \'CAUTION\' sign.",\n        "position": "On a stand on the left grass verge."\n      },\n      {\n        "type": "sign",\n        "characteristics": "A circular \'No Pedestrian Crossing\' sign.",\n        "position": "On a pole on the left side of the road."\n      },\n      {\n        "type": "sign",\n        "characteristics": "A triangular \'Give Way\' sign.",\n        "position": "On a pole on a traffic island to the right."\n      },\n      {\n        "type": "sign",\n        "characteristics": "A circular blue \'Roundabout\' sign.",\n        "position": "On a pole on a traffic island to the right."\n      },\n      {\n        "type": "sign",\n        "characteristics": "A blue rectangular \'End Of School Zone\' sign.",\n        "position": "On a pole on the left side of the road."\n      },\n      {\n        "type": "sign",\n        "characteristics": "A yellow triangular warning sign for \'School Zone\' with a blue rectangular sign below it.",\n        "position": "On a pole on the left side of the road."\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Green traffic light for straight traffic.",\n        "position": "Ahead at the intersection, suspended over the road."\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "vegetation",\n        "characteristics": "Numerous large, leafy trees and dense green hedges line both sides of the road.",\n        "position": "On the left and right sides of the road."\n      },\n      {\n        "type": "multy-story building",\n        "characteristics": "Tall residential apartment buildings are visible in the background behind the trees.",\n        "position": "In the distance on both the left and right."\n      },\n      {\n        "type": "wall or barrier",\n        "characteristics": "Green metal fence running along the roadside.",\n        "position": "Along the right side of the road."\n      },\n      {\n        "type": "small building",\n        "characteristics": "A covered bus stop shelter.",\n        "position": "On the right side of the road at the start of the video."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Grey metal streetlight poles.",\n        "position": "Periodically along both sides of the road."\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "Temporary Signage",\n        "characteristic": "A portable orange \'CAUTION\' sign on a metal stand.",\n        "position": "On the grass verge to the left of the road."\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "pedestrian",\n        "characteristic": "Two people walking on the sidewalk.",\n        "position": "On the sidewalk to the left of the ego vehicle.",\n        "motion_type": "moving toward ego",\n        "motion_characteristic": "Walking at a casual pace along the sidewalk."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Dark colored SUV.",\n        "position": "Far ahead of the ego vehicle in the same lane.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving straight ahead."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Silver sedan.",\n        "position": "Ahead of the ego vehicle in the right lane.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving straight ahead."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "White van.",\n        "position": "Far ahead of the ego vehicle.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Driving straight ahead."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Silver sedan.",\n        "position": "On the right side of the road, appearing to wait at an intersection.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Stationary at a side road junction."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Motorcycle.",\n        "position": "On the left side of the road, emerging from a driveway or side road.",\n        "motion_type": "other",\n        "motion_characteristic": "Turning right to join the main road."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Blue taxi.",\n        "position": "In the lane to the right of the ego vehicle.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Overtaking the ego vehicle on the right side."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "The sky is blue with a few scattered white clouds."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Bright daylight, causing strong, distinct shadows from trees and buildings to fall across the road."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "road",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 3.0,\n        "surface_material": "asphalt",\n        "eventual_damages": "The road is damp with some patches."\n      },\n      {\n        "type": "bus_lane",\n        "geometry": "straight",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 1,\n        "lane_size": 3.0,\n        "surface_material": "asphalt",\n        "eventual_damages": "None observed."\n      },\n      {\n        "type": "parking",\n        "geometry": "straight",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 0,\n        "lane_size": 2.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "None observed."\n      },\n      {\n        "type": "road",\n        "geometry": "turning left",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 2,\n        "lane_size": 3.0,\n        "surface_material": "asphalt",\n        "eventual_damages": "The road is damp with some patches."\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "Yellow box marking for a bus stop.",\n        "position": "On the road surface on the right side from 00:02 to 00:05."\n      },\n      {\n        "type": "marking",\n        "characteristics": "White arrow painted on the road, indicating the direction of travel.",\n        "position": "On the road surface in the center of the lane from 00:09 onwards."\n      },\n      {\n        "type": "marking",\n        "characteristics": "Red lines and text indicating reserved parking spaces.",\n        "position": "On the road surface along the sides of the road where cars are parked."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Blue sign indicating a bus stop.",\n        "position": "On a pole next to the bus stop on the right side of the road at 00:03."\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "multy-story building",\n        "characteristics": "A large, modern concrete building with multiple floors and large windows, likely an academic or office building.",\n        "position": "On the left side of the scene from 00:00 to 00:02."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "A covered bus stop shelter with a metal frame and transparent panels.",\n        "position": "On the right side of the road from 00:01 to 00:04."\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "Numerous large, leafy trees, palm trees, and dense bushes.",\n        "position": "Lining both sides of the road throughout the entire video."\n      },\n      {\n        "type": "wall or barrier",\n        "characteristics": "Metal guardrail with black and white striped posts.",\n        "position": "On the left side of the road along the curve from 00:19 to 00:20."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Tall, metal streetlights.",\n        "position": "Positioned along the road at various intervals throughout the video."\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "Water on road",\n        "characteristic": "The asphalt surface is damp in many areas, indicating recent rain or high humidity.",\n        "position": "Intermittently on the road surface throughout the video."\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "pedestrian",\n        "characteristic": "A person wearing a red shirt and dark pants.",\n        "position": "Walking from right to left in front of the ego vehicle at 00:00.",\n        "motion_type": "moving toward ego",\n        "motion_characteristic": "Walking at a normal pace across the road."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "Two people wearing casual clothing, one with a backpack.",\n        "position": "Walking on the left side of the road from 00:00 to 00:02.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Walking at a normal pace."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "A group of people waiting.",\n        "position": "Under the bus stop shelter on the right side from 00:01 to 00:04.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Standing and waiting for a bus."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "A person wearing a light blue shirt and khaki pants.",\n        "position": "Walking on the right side of the road from 00:03 to 00:05.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Walking at a normal pace."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple motorcycles and scooters of various colors.",\n        "position": "Parked on the right side of the road near the bus stop from 00:03 to 00:06.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Stationary in parking spots."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple cars of various types (sedans, SUVs, hatchbacks) and colors (blue, silver, white, grey).",\n        "position": "Parked along both sides of the road from 00:04 onwards.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Stationary in designated and undesignated parking spots."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "A white light-duty truck.",\n        "position": "Parked on the left side of the road from 00:10 to 00:15.",\n        "motion_type": "not moving",\n        "motion_characteristic": "Stationary."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "cloudy",\n      "characteristics": "The sky is completely overcast with grey clouds. The road surface is damp, suggesting it rained recently."\n    },\n    "illumination": [\n      {\n        "light_source": "sunlight",\n        "characteristic": "Daytime, but the light is dim and diffuse due to heavy cloud cover. Shadows are very soft or non-existent."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 8,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "The road surface is wet and reflective."\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "White dashed lines separating the lanes.",\n        "position": "On the road surface between lanes."\n      },\n      {\n        "type": "marking",\n        "characteristics": "Solid white line marking the edge of the lane.",\n        "position": "On the right edge of the ego vehicle\'s lane."\n      },\n      {\n        "type": "marking",\n        "characteristics": "Double solid yellow lines.",\n        "position": "On the left edge of the ego vehicle\'s lane."\n      },\n      {\n        "type": "sign",\n        "characteristics": "Speed limit sign, appears to be \'30\'.",\n        "position": "On a pole on the left side of the road."\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "vegetation",\n        "characteristics": "Dense trees and bushes.",\n        "position": "Lining the left side of the road and in the median strip to the right."\n      },\n      {\n        "type": "wall or barrier",\n        "characteristics": "Low concrete kerb painted with black and white stripes.",\n        "position": "On both the left and right sides of the ego vehicle\'s road."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "Streetlight poles.",\n        "position": "Periodically along both sides of the road."\n      },\n      {\n        "type": "skyscraper",\n        "characteristics": "Tall, illuminated buildings.",\n        "position": "Visible in the distant background."\n      },\n      {\n        "type": "miscellaneous man-made structure",\n        "characteristics": "A bus stop shelter with internal lighting.",\n        "position": "On the left side of the road."\n      }\n    ]\n  },\n  "L3": {\n    "change_list": []\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Dark-colored hatchback or MPV.",\n        "position": "Directly in front of the ego vehicle, in the same lane.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Moving at a consistent speed, maintaining distance."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple cars of various types (sedans, SUVs).",\n        "position": "On the adjacent highway to the right of the ego vehicle.",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Flowing traffic moving at a similar speed to the ego vehicle."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "Nighttime. The road is wet, indicating recent rainfall."\n    },\n    "illumination": [\n      {\n        "light_source": "lamps",\n        "characteristic": "Bright streetlights are the primary source of illumination, causing some lens flare and glare."\n      },\n      {\n        "light_source": "vehicle_light",\n        "characteristic": "Headlights and taillights from other vehicles contribute to the scene\'s illumination."\n      },\n      {\n        "light_source": "reflection",\n        "characteristic": "Strong reflections from all light sources are visible on the wet asphalt surface."\n      }\n    ]\n  }\n}',
 '{\n  "L1": {\n    "roads": [\n      {\n        "type": "highway",\n        "geometry": "straight",\n        "main_road": true,\n        "total_number_of_lanes_in_both_direction": 6,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "wet surface"\n      },\n      {\n        "type": "road",\n        "geometry": "turning left",\n        "main_road": false,\n        "total_number_of_lanes_in_both_direction": 4,\n        "lane_size": 3.5,\n        "surface_material": "asphalt",\n        "eventual_damages": "wet surface"\n      }\n    ],\n    "guidance_objects": [\n      {\n        "type": "marking",\n        "characteristics": "White and yellow lane dividing lines.",\n        "position": "on the road surface"\n      },\n      {\n        "type": "marking",\n        "characteristics": "White stop line for the intersecting road.",\n        "position": "on the intersecting road surface"\n      },\n      {\n        "type": "sign",\n        "characteristics": "Blue rectangular directional sign.",\n        "position": "on a pole on the right side of the road"\n      },\n      {\n        "type": "traffic light",\n        "characteristics": "Traffic light showing green for the intersecting traffic.",\n        "position": "on the right side of the intersection"\n      }\n    ]\n  },\n  "L2": {\n    "environment": "urban",\n    "structures": [\n      {\n        "type": "skyscraper",\n        "characteristics": "Tall, illuminated office or residential buildings.",\n        "position": "in the background"\n      },\n      {\n        "type": "bridge",\n        "characteristics": "A pedestrian overpass with metal railings and vegetation.",\n        "position": "spanning over the road ahead"\n      },\n      {\n        "type": "small building",\n        "characteristics": "A covered walkway or bus stop structure.",\n        "position": "on the right sidewalk"\n      },\n      {\n        "type": "vegetation",\n        "characteristics": "Trees and bushes.",\n        "position": "on the road medians and alongside the bridge"\n      },\n      {\n        "type": "wall or barrier",\n        "characteristics": "Concrete curbs painted black and white.",\n        "position": "along the edges of the road and medians"\n      }\n    ]\n  },\n  "L3": {\n    "change_list": [\n      {\n        "type": "Temporary road sign",\n        "characteristic": "A blue circular sign with a white arrow pointing left, mounted on a portable stand.",\n        "position": "on the road median to the left"\n      }\n    ]\n  },\n  "L4": {\n    "object_list": [\n      {\n        "type": "Vehicle",\n        "characteristic": "Multiple dark-colored cars.",\n        "position": "on the right side of the intersection, turning left in front of the ego vehicle",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Continuously turning left from the intersecting road throughout the video."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Red and white public bus.",\n        "position": "in the far left lane, ahead of the ego vehicle",\n        "motion_type": "not moving",\n        "motion_characteristic": "Waiting at the intersection."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "Several cars of various colors.",\n        "position": "in the lanes to the left of the ego vehicle",\n        "motion_type": "not moving",\n        "motion_characteristic": "Waiting at the intersection."\n      },\n      {\n        "type": "Vehicle",\n        "characteristic": "A motorcycle or scooter.",\n        "position": "crossing the intersection from right to left",\n        "motion_type": "moving in same direction as ego",\n        "motion_characteristic": "Turning left from the intersecting road."\n      },\n      {\n        "type": "pedestrian",\n        "characteristic": "A person standing on the sidewalk.",\n        "position": "on the far right side of the road",\n        "motion_type": "not moving",\n        "motion_characteristic": "Standing still."\n      },\n      {\n        "type": "other",\n        "characteristic": "A large, semi-transparent, blueish visual artifact, likely an internal reflection or lens flare.",\n        "position": "in the center of the video frame",\n        "motion_type": "not moving",\n        "motion_characteristic": "Static relative to the camera frame."\n      }\n    ]\n  },\n  "L5": {\n    "weather": {\n      "type": "clear",\n      "characteristics": "The sky is clear but the road surface is wet, indicating recent rainfall."\n    },\n    "illumination": [\n      {\n        "light_source": "lamps",\n        "characteristic": "Bright overhead streetlights illuminating the entire scene."\n      },\n      {\n        "light_source": "vehicle_light",\n        "characteristic": "Headlights and taillights from numerous vehicles on the road."\n      },\n      {\n        "light_source": "neon",\n        "characteristic": "Lights from the windows of skyscrapers in the background."\n      },\n      {\n        "light_source": "reflection",\n        "characteristic": "Strong reflections of all light sources on the wet asphalt."\n      },\n      {\n        "light_source": "other",\n        "characteristic": "Light from traffic signals."\n      }\n    ]\n  }\n}']

## Example of component diversity calculation for the reference scenes:

In [ ]:
structured_ref_scene_layers4 = [json.loads(ref)['L4'] for ref in structured_ref_scene_2]
nb = []
for L in structured_ref_scene_layers4:
    nb.append(len(L['object_list']))
    print(nb[-1])
print(f'average nb of object: {np.average(nb)}')
custom_diversity(structured_ref_scene_layers4, 'object_list', Dynamic_object_type)

In [ ]:
 # Choose the layer to evaluate. Note that L3 is not applicable in the current implementation
custom_component_metrics(
    "Path/to/experiment/results",
    'L1',
    metric= 'div')